## Exercise 1 - Time Until 5 pm tomorrow
### 1 - Corner cases when calculating time to 5PM tomorrow
 - Tomorrow is unknown (Feb 28 on an unknown year)
 - DST forward, DST backward, start or land mid-fold
 - Run program at midnight, date changes mid-execution
 - Tomorrow is calculated correctly (skipping over a 23 hour day)
 
 ### 2 - The `time_till_5pm_tomorrow` function

In [1]:
from datetime import datetime, timedelta, date, time, timezone
import copy

In [2]:
def time_till_5pm_tomorrow(now):
    tomorrow = now.date() + timedelta(days=1)
    fivepm = time(hour=17, tzinfo = now.tzinfo)
    tomorrow_fivepm = datetime.combine(tomorrow, fivepm)
    seconds_till = tomorrow_fivepm.timestamp() - now.timestamp()
    return seconds_till/3600

### 3 - Testing `time_till_5pm_tomorrow`

In [3]:
# baseline test cases, run in US Eastern Time
# print("expected anwser: unknown")
# print(time_till_5pm_tomorrow())

print("expected anwser: 24.0")
print(time_till_5pm_tomorrow(datetime(2024,6,18,17)))

expected anwser: 24.0
24.0


In [4]:
# DST test cases, run in US Eastern Time

# March 9-10, the start of DST
print("expected anwser: 23.0")
print(time_till_5pm_tomorrow(datetime(2024,3,9,17)))

# November 2-3, the end of DST
print("expected anwser: 25.0")
print(time_till_5pm_tomorrow(datetime(2024,11,2,17)))

# March 9 11:30pm, 30 minutes before DST
print("expected anwser: 16.5")
print(time_till_5pm_tomorrow(datetime(2024,3,9,23,30)))


expected anwser: 23.0
23.0
expected anwser: 25.0
25.0
expected anwser: 16.5
16.5


### 4 - Using timezone libraries
I'm using each libraries closest approximation of NYC time, and running the last four tests which should get answers: 24, 23, 25, and 16.5

In [5]:
import pytz, dateutil
from zoneinfo import ZoneInfo

In [6]:
# datetime.timezone on EST (UTC-5:00)
NYC = timezone(-timedelta(hours=5), name="America/New_York")
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
24.0
24.0
17.5


In [7]:
# pytz.timezone('America/New_York') run naively
NYC = pytz.timezone('America/New_York')
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
24.0
24.0
17.5


In [33]:
# pytz.timezone('America/New_York') run corrctly

def time_till_5pm_tomorrow_pytz(now):
    tz = now.tzinfo
    tomorrow = now.date() + timedelta(days=1)
    tomorrow_fivepm = tz.localize(datetime.combine(tomorrow, time(hour=17)))
    seconds_till = tomorrow_fivepm.timestamp() - now.timestamp()
    return seconds_till/3600

NYC = pytz.timezone('America/New_York')
print(time_till_5pm_tomorrow_pytz(NYC.localize(datetime(2024,6, 18,17))))
print(time_till_5pm_tomorrow_pytz(NYC.localize(datetime(2024,3, 9, 17))))
print(time_till_5pm_tomorrow_pytz(NYC.localize(datetime(2024,11,2, 17))))
print(time_till_5pm_tomorrow_pytz(NYC.localize(datetime(2024,3, 9, 23, 30))))

24.0
23.0
25.0
16.5
39.0


In [9]:
# dateutil.tz.gettz('America/New_York')
NYC = dateutil.tz.gettz('America/New_York')
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
23.0
25.0
16.5


In [10]:
# zoneinfo.ZoneInfo('America/New_York')
NYC = ZoneInfo('America/New_York')
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
23.0
25.0
16.5


### 4.5 - Conclusions
The only two that work "out of the box" are dateutil.tz and ZoneInfo. datetime.timezone doesn't support DST at all and pytz requires some major changes to the fucnction structure for it to work correctly.

### 5 - Comparing pytz and dateutil.tz
This [ganssle blog](https://blog.ganssle.io/articles/2018/03/pytz-fastest-footgun.html) summarizes it pretty well. pytz attempts to resolve to a single timezone whenever possible and so needs to be "prompted" with normalize and localize to make dynamic changes like DST. dateutil.tz on the other hand, is lazy evaluated and so can check what date it

### 6 - Trying out different libraries
I'm using the same tests with local time, zoneinfo and dateutil.tz. If there's another built in way to handle time zones in the package, I'll also test that.

### 6.1 - Arrow
Unlike the main library, it assumes that unspecified time zones are UTC. The built in time zones work on these tests, and it also interfaces correctly with the other two timezone packages. Stuff like shift and floor are handy and it looks like theres a bunch of other useful methods.

In [11]:
import arrow
def time_till_5pm_tomorrow(now=None):
    if now is None:
        now = arrow.now()
    tomorrow = now.shift(days = 1)
    tomorrow_fivepm = tomorrow.replace(hour=17).floor('hour')
    seconds_till = tomorrow_fivepm.timestamp() - now.timestamp()
    return seconds_till/3600

In [12]:
# print(time_till_5pm_tomorrow())
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30)))
print("-----")
NYC = 'America/New_York'
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30,tzinfo=NYC)))
print("-----")
NYC = dateutil.tz.gettz('America/New_York')
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30,tzinfo=NYC)))
print("-----")
NYC = ZoneInfo('America/New_York')
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30,tzinfo=NYC)))

24.0
24.0
24.0
17.5
-----
24.0
23.0
25.0
16.5
-----
24.0
23.0
25.0
16.5
-----
24.0
23.0
25.0
16.5


### 6.2 - pendulum
Also defaults to UTC (but supports naives if you need them). Similarly helpful stuff to arrow. Doesn't play well with dateutil.tz for some reason. 

In [13]:
import pendulum
def time_till_5pm_tomorrow(now=None):
    if now is None:
        now = pendulum.now()
    tomorrow = now.add(days = 1)
    tomorrow_fivepm = tomorrow.replace(hour=17).start_of('hour')
    seconds_till = tomorrow_fivepm.timestamp() - now.timestamp()
    return seconds_till/3600

In [14]:
# print(time_till_5pm_tomorrow())
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30)))
print("-----")
NYC = 'America/New_York'
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30,tz=NYC)))
print("-----")
NYC = dateutil.tz.gettz('America/New_York')
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30,tz=NYC)))
print("-----")
NYC = ZoneInfo('America/New_York')
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30,tz=NYC)))

24.0
24.0
24.0
17.5
-----
24.0
23.0
25.0
16.5
-----
24.0
24.0
24.0
17.5
-----
24.0
23.0
25.0
16.5


### 6.3 - whenever
I'm a huge fan of type safety (which is why I'm working with python less and less these days), so I like the philosophy. It feels like because the main focus is safety, it has less useful methods than the other two like `humanize` or `floor`. It doesn't interface with any other timezone package, but that's because it's rebuilding the system from the ground up, which is probably a good thing.

In [15]:
import whenever
def time_till_5pm_tomorrow(now=None):
    if now is None:
        now = whenever.UTCDateTime.now()
    tomorrow = now + whenever.days(1)
    tomorrow_fivepm = tomorrow.replace(hour=17, minute=0, second=0, microsecond=0)
    diff = tomorrow_fivepm - now
    return diff.in_hours()

In [16]:
# print(time_till_5pm_tomorrow())
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,6 ,18,17,   tz = 'America/New_York')))
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,3 ,9 ,17,   tz = 'America/New_York')))
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,11,2 ,17,   tz = 'America/New_York')))
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,3 ,9 ,23,30,tz = 'America/New_York')))

24.0
23.0
25.0
16.5


# Exercise 2 - Date arithmetic
### 1 - Edge cases (+ my expected behavior)
 - Add 1 year to Feb 29
   - Feb 28
 - Add 1 month to Jan 31
   - Feb 28/29
 - Add 1 hour to 1:30 on the end of DST (first, second or unspecified fold)
   - default to first, first -> second, second -> 2:30
 - Add 1 day to 1:30 on the end of DST (first and second fold)
   - Just do wall clock time (so 1:30 next day in either case)
 - Add or remove 1 day to land at 1:30 on the end of DST
   - I would expect first fold, but I think the RFC says to do the furthest fold, so second when moving forward and first when moving backward.
 - Add 1 month, 2 days to Jan 29 (order of operations)
   - Mar 2 (larger time periods first)
 - (1 day, 15 hours) / 2
   - either invalid or 19.5 hours
 - (1 day, 15 hours) * 2
   - 3 days, 6 hours
 - difference over DST (both ways)
   - no clue what I'd expect honestly. I think I agree with [this blog](https://blog.ganssle.io/articles/2018/02/aware-datetime-arithmetic.html) that it should be absolute time, but I could see either honestly.
 - difference across time zones
   - absolute time

### 2 - Testing
Some of these tests don't make sense with the default datetime library, but of the ones that do, most of them work as expected. The main notable behavior is:
 - In test three, adding an hour to the first fold skips over the second fold entirely. I think this is an issue with backwards compatibility, so they can't fix the incorrect behavior.
 - Test five always lands in the first fold.
 - Tests nine records wall clock time, which is not what I would personally choose, but makes sense.

In [17]:
# 1,2
# Timedelta doesn't work with years and months, so I'm seeing what happens if you just manually edit the month and year
try:
    d = date(2024,2,29)
    print(d)
    print(d.replace(year = d.year + 1))
except Exception as e:
    print(e)

try:
    d = date(2024,1,31)
    print(d)
    print(d.replace(month = d.month + 1))
except Exception as e:
    print(e)

2024-02-29
day is out of range for month
2024-01-31
day is out of range for month


In [18]:
# 3,4
NYC = dateutil.tz.gettz('America/New_York')
d0 = datetime(2024,11,3,1,30, tzinfo = NYC)
d1 = datetime(2024,11,3,1,30, fold = 0, tzinfo = NYC)
d2 = datetime(2024,11,3,1,30, fold = 1, tzinfo = NYC)

print("d0 =", d0, "=", d0.timestamp())
print("d1 =", d1, "=", d1.timestamp())
print("d2 =", d2, "=", d2.timestamp())
# Note: does default to fold = 0

print(d1 + timedelta(hours = 1))
print(d1 + timedelta(days = 1))
# unexpected behavior, skips over the extra daylight savings hour

print(d2 + timedelta(hours = 1))
print(d2 + timedelta(days = 1))

d0 = 2024-11-03 01:30:00-04:00 = 1730611800.0
d1 = 2024-11-03 01:30:00-04:00 = 1730611800.0
d2 = 2024-11-03 01:30:00-05:00 = 1730615400.0
2024-11-03 02:30:00-05:00
2024-11-04 01:30:00-05:00
2024-11-03 02:30:00-05:00
2024-11-04 01:30:00-05:00


In [19]:
# 5
NYC = dateutil.tz.gettz('America/New_York')
d = datetime(2024,11,4,1,30, tzinfo = NYC)
print(d + timedelta(days = -1))
d = datetime(2024,11,2,1,30, tzinfo = NYC)
print(d + timedelta(days = 1))

2024-11-03 01:30:00-04:00
2024-11-03 01:30:00-04:00


In [20]:
# 6
# invalid because the default datetime library doesn't support adding months

In [21]:
# 7, 8
td = timedelta(days=1, hours=15)
print(td)
print(td*2)
print(td/2)
print(td//2)

1 day, 15:00:00
3 days, 6:00:00
19:30:00
19:30:00


In [22]:
# 9
NYC = dateutil.tz.gettz('America/New_York')
d1 = datetime(2024,11,2,12, tzinfo = NYC)
d2 = datetime(2024,11,3,12, tzinfo = NYC)
print(d2-d1)

NYC = dateutil.tz.gettz('America/New_York')
d1 = datetime(2024,3,9,12, tzinfo = NYC)
d2 = datetime(2024,3,10,12, tzinfo = NYC)
print(d2-d1)

1 day, 0:00:00
1 day, 0:00:00


In [23]:
#10
d1 = datetime(2024,1,1, tzinfo = dateutil.tz.gettz('America/Los_Angeles'))
d2 = datetime(2024,1,1, tzinfo = dateutil.tz.gettz('America/New_York'))
print(d1-d2)

3:00:00


### 2.1 - Package arithmetic
Since the different datetime packages handle arithmetic a little bit differently, I'm going to run an abbreviated test on them. All the actual tests are below, but I'll also put a quick summary of the notable results up here.

**DateTime and relativedelta:**
 - 1 and 2 work as expected
 - 3 still skips over the second fold 
 - 6 works as expected
 - 7 does some really agressive rounding.

**Arrow:**
 - 1 and 2 work as expected
 - all other tests work identical to datetime (notably 3 is still weird)

**Pendulum:**
 - 1 and 2 work as expected
 - In ambiguous cases, it defaults to the second fold
 - 3 works as expected on both folds
 - 9 records absolute time

**Whenver:**
 - 1 and 2 work as expected
 - Not providing a fold will throw an error sometimes, but other than that it defaults to first fold
 - 6 is a bit funky to set up because of how whenever likes to do deltas, but it works as expected
 - 7 throws an error
 - 9 records absolute time

In [24]:
# datetime + dateutil.relativedelta
from dateutil.relativedelta import relativedelta
NYC = dateutil.tz.gettz('America/New_York')

print("1:", datetime(2024, 2, 29) + relativedelta(years=1))
print("2:", datetime(2023, 1, 31) + relativedelta(months=1)) # 2024 is a leap year, so gotta use 2023
print("3:", datetime(2024,11,3,1,30, fold=0, tzinfo = NYC) + relativedelta(hours=1)) # still skips 
print("3:", datetime(2024,11,3,1,30, fold=1, tzinfo = NYC) + relativedelta(hours=1))
print("4:", datetime(2024,11,3,1,30, fold=0, tzinfo = NYC) + relativedelta(days=1))
print("4:", datetime(2024,11,3,1,30, fold=1, tzinfo = NYC) + relativedelta(days=1))
print("5:", datetime(2024,11,4,1,30, tzinfo = NYC) - relativedelta(days=1)) # lands in first fold
print("5:", datetime(2024,11,2,1,30, tzinfo = NYC) + relativedelta(days=1)) # lands in first fold
print("6:", datetime(2023, 1, 29) + relativedelta(months=1, days=2))
print("7:", relativedelta(days=1, hours=15) / 2)
print("8:", relativedelta(days=1, hours=15) * 2)
# 9 and 10 will be the same as datetime since they don't use relative delta

1: 2025-02-28 00:00:00
2: 2023-02-28 00:00:00
3: 2024-11-03 02:30:00-05:00
3: 2024-11-03 02:30:00-05:00
4: 2024-11-04 01:30:00-05:00
4: 2024-11-04 01:30:00-05:00
5: 2024-11-03 01:30:00-04:00
5: 2024-11-03 01:30:00-04:00
6: 2023-03-02 00:00:00
7: relativedelta(hours=+7)
8: relativedelta(days=+3, hours=+6)


In [25]:
# Arrow
import arrow
NYC = 'America/New_York'

print("1:", arrow.Arrow(2024, 2, 29).shift(years=1))
print("2:", arrow.Arrow(2023, 1, 31).shift(months=1)) # 2024 is a leap year, so gotta use 2023
# Note: defaults to second fold
print("3:", arrow.Arrow(2024,11,3,1,30, tzinfo = NYC)) # defaults to first fold
print("3:", arrow.Arrow(2024,11,3,1,30, fold=0, tzinfo = NYC).shift(hours=1)) # also skips fold 2 like datetime
print("3:", arrow.Arrow(2024,11,3,1,30, fold=1, tzinfo = NYC).shift(hours=1))
print("4:", arrow.Arrow(2024,11,3,1,30, fold=0, tzinfo = NYC).shift(days=1))
print("4:", arrow.Arrow(2024,11,3,1,30, fold=1, tzinfo = NYC).shift(days=1))
print("5:", arrow.Arrow(2024,11,4,1,30, tzinfo = NYC).shift(days=-1)) # lands in first fold
print("5:", arrow.Arrow(2024,11,2,1,30, tzinfo = NYC).shift(days=1)) # lands in first fold
print("6:", arrow.Arrow(2023, 1, 29).shift(months=1, days=2))
# arrow uses the default datetime timedeltas, so 7 and 8 are the same as default
print("9:", arrow.Arrow(2024,11,3,12, tzinfo = NYC) - arrow.Arrow(2024,11,2,12, tzinfo = NYC))
print("9:", arrow.Arrow(2024,3,10,12, tzinfo = NYC) - arrow.Arrow(2024,3,9,12, tzinfo = NYC))
print("10:", arrow.Arrow(2024,1,1, tzinfo = 'America/Los_Angeles') - arrow.Arrow(2024,1,1, tzinfo = 'America/New_York'))

1: 2025-02-28T00:00:00+00:00
2: 2023-02-28T00:00:00+00:00
3: 2024-11-03T01:30:00-04:00
3: 2024-11-03T02:30:00-05:00
3: 2024-11-03T02:30:00-05:00
4: 2024-11-04T01:30:00-05:00
4: 2024-11-04T01:30:00-05:00
5: 2024-11-03T01:30:00-04:00
5: 2024-11-03T01:30:00-04:00
6: 2023-03-02T00:00:00+00:00
9: 1 day, 0:00:00
9: 1 day, 0:00:00
10: 3:00:00


In [26]:
# Pendulum
import pendulum
NYC = 'America/New_York'

print("1:", pendulum.datetime(2024, 2, 29).add(years=1))
print("2:", pendulum.datetime(2023, 1, 31).add(months=1))
print("3:", pendulum.datetime(2024,11,3,1,30, tz = NYC)) # defaults to second fold
print("3:", pendulum.datetime(2024,11,3,1,30, fold=0, tz = NYC).add(hours=1))
print("3:", pendulum.datetime(2024,11,3,1,30, fold=1, tz = NYC).add(hours=1))
print("4:", pendulum.datetime(2024,11,3,1,30, fold=0, tz = NYC).add(days=1))
print("4:", pendulum.datetime(2024,11,3,1,30, fold=1, tz = NYC).add(days=1))
print("5:", pendulum.datetime(2024,11,4,1,30, tz = NYC).add(days=-1)) # lands in second fold
print("5:", pendulum.datetime(2024,11,2,1,30, tz = NYC).add(days=1)) # lands in second fold
print("6:", pendulum.datetime(2023, 1, 29).add(months=1, days=2)) # expected behavior
# pendulum does not have timedeltas and so 7 and 8 don't apply
print("9:", (pendulum.datetime(2024,11,3,12, tz = NYC) - pendulum.datetime(2024,11,2,12, tz = NYC)).in_hours())
print("9:", (pendulum.datetime(2024,3,10,12, tz = NYC) - pendulum.datetime(2024,3,9,12, tz = NYC)).in_hours())
print("10:", (pendulum.datetime(2024,1,1, tz = 'America/Los_Angeles') - pendulum.datetime(2024,1,1, tz = 'America/New_York')).in_hours())

1: 2025-02-28 00:00:00+00:00
2: 2023-02-28 00:00:00+00:00
3: 2024-11-03 01:30:00-05:00
3: 2024-11-03 01:30:00-05:00
3: 2024-11-03 02:30:00-05:00
4: 2024-11-04 01:30:00-05:00
4: 2024-11-04 01:30:00-05:00
5: 2024-11-03 01:30:00-05:00
5: 2024-11-03 01:30:00-05:00
6: 2023-03-02 00:00:00+00:00
9: 25
9: 23
10: 3


In [27]:
# Whenever
import whenever
NYC = 'America/New_York'
# ZonedDateTime
print("1:", whenever.NaiveDateTime(2024, 2, 29) + whenever.years(1))
print("2:", whenever.NaiveDateTime(2023, 1, 31) + whenever.months(1))
print("3:", whenever.ZonedDateTime(2024,11,3,1,30, disambiguate='compatible', tz = NYC))
# by default throws an error when trying to initialize an abiguous time
print("3:", whenever.ZonedDateTime(2024,11,3,1,30, disambiguate='earlier', tz = NYC) + whenever.hours(1))
print("3:", whenever.ZonedDateTime(2024,11,3,1,30, disambiguate='later', tz = NYC) + whenever.hours(1))
print("4:", whenever.ZonedDateTime(2024,11,3,1,30, disambiguate='earlier', tz = NYC) + whenever.days(1))
print("4:", whenever.ZonedDateTime(2024,11,3,1,30, disambiguate='later', tz = NYC) + whenever.days(1))
print("5:", whenever.ZonedDateTime(2024,11,4,1,30, tz = NYC) - whenever.days(1)) # lands in the first fold
print("5:", whenever.ZonedDateTime(2024,11,2,1,30, tz = NYC) + whenever.days(1))
print("6:", whenever.NaiveDateTime(2023, 1, 29) + (whenever.months(1) + whenever.days(2)))
print("6:", whenever.NaiveDateTime(2023, 1, 29) + (whenever.days(2) + whenever.months(1)))
print("6:", whenever.NaiveDateTime(2023, 1, 29) + whenever.days(2) + whenever.months(1))
# print("7:", (whenever.days(1) + whenever.hours(15)) / 2) # not allowed to divide
print("8:", (whenever.days(1) + whenever.hours(15)) * 2)
print("9:", whenever.ZonedDateTime(2024,11,3,12, tz = NYC) - whenever.ZonedDateTime(2024,11,2,12, tz = NYC))
print("9:", whenever.ZonedDateTime(2024,3,10,12, tz = NYC) - whenever.ZonedDateTime(2024,3,9,12, tz = NYC))
print("10:", whenever.ZonedDateTime(2024,1,1, tz = 'America/Los_Angeles') - whenever.ZonedDateTime(2024,1,1, tz = 'America/New_York'))

1: 2025-02-28 00:00:00
2: 2023-02-28 00:00:00
3: 2024-11-03 01:30:00-04:00[America/New_York]
3: 2024-11-03 01:30:00-05:00[America/New_York]
3: 2024-11-03 02:30:00-05:00[America/New_York]
4: 2024-11-04 01:30:00-05:00[America/New_York]
4: 2024-11-04 01:30:00-05:00[America/New_York]
5: 2024-11-03 01:30:00-04:00[America/New_York]
5: 2024-11-03 01:30:00-04:00[America/New_York]
6: 2023-03-02 00:00:00
6: 2023-03-02 00:00:00
6: 2023-02-28 00:00:00
8: P2DT30H
9: 25:00:00
9: 23:00:00
10: 03:00:00


### 3 - Equality
mostly works how you expect, except that same-time zone equality ignores folds. Notably using an equivalent but not equal (in this case copy.deepcopy) timezone makes it care about folds (because it converts into UTC instead of comparing wall clock time).

In [31]:
NYC = dateutil.tz.gettz('America/New_York')
NYC1 = copy.deepcopy(NYC)
LAX = dateutil.tz.gettz('America/Los_Angeles')
print("same time, same zone:", datetime(2024,6,20,10,13,42,tzinfo=NYC) == datetime(2024,6,20,10,13,42,tzinfo=NYC))
print("same wall clock, different zone:", datetime(2024,6,20,10,13,42,tzinfo=NYC) == datetime(2024,6,20,10,13,42,tzinfo=LAX))
print("same time, different zone:", datetime(2024,6,20,10,13,42,tzinfo=NYC) == datetime(2024,6,20,7,13,42,tzinfo=LAX))
print("same time, same zone, different fold:", datetime(2024,11,3,1,30, fold = 0, tzinfo = NYC) == datetime(2024,11,3,1,30, fold = 1, tzinfo = NYC))
print("same time, equal zone, different fold:", datetime(2024,11,3,1,30, fold = 0, tzinfo = NYC) == datetime(2024,11,3,1,30, fold = 1, tzinfo = NYC1))

same time, same zone: True
same wall clock, different zone: False
same time, different zone: True
same time, same zone, different fold: True
same time, equal zone, different fold: False


### 3.1 - Package Equality
I'm just going to check how each one deals with folds, because I assume that the rest works correctly. Of the three only Whenever catches this bug. If we use equivalent zones (in this case a string and a zoneinfo), Arrow says false, much like the builtin library, and pendulum says true, which is consistent but a bit strange.

In [29]:
NYC = 'America/New_York'
print("Arrow:", arrow.Arrow(2024,11,3,1,30, fold = 0, tzinfo=NYC) == arrow.Arrow(2024,11,3,1,30, fold = 1, tzinfo=NYC))
print("Pendulum:", pendulum.datetime(2024,11,3,1,30, fold = 0, tz=NYC) == pendulum.datetime(2024,11,3,1,30, fold = 1, tz=NYC))
print("Whenever:", whenever.ZonedDateTime(2024,11,3,1,30, disambiguate='earlier', tz=NYC) == whenever.ZonedDateTime(2024,11,3,1,30, disambiguate='later', tz=NYC))

NYCZI = ZoneInfo('America/New_York')
print("Arrow:", arrow.Arrow(2024,11,3,1,30, fold = 0, tzinfo=NYC) == arrow.Arrow(2024,11,3,1,30, fold = 1, tzinfo=NYCZI))
print("Pendulum:", pendulum.datetime(2024,11,3,1,30, fold = 0, tz=NYC) == pendulum.datetime(2024,11,3,1,30, fold = 1, tz=NYCZI))

Arrow: True
Pendulum: True
Whenever: False
Arrow: False
Pendulum: True


### 4 - My Equality Operator
I think the most reasonable way to decide if two times are equal is to check UTC timestamps. The main case where this might be odd is for naive times, where I'll assume they're in local time (which is the default behavior of `.timestamp()`).

In [30]:
def dt_eq(t1, t2):
    return (t1.timestamp() == t2.timestamp())

NYC = dateutil.tz.gettz('America/New_York')
NYC1 = copy.deepcopy(NYC)
LAX = dateutil.tz.gettz('America/Los_Angeles')
print("same time, same zone:", dt_eq(datetime(2024,6,20,10,13,42,tzinfo=NYC), datetime(2024,6,20,10,13,42,tzinfo=NYC)))
print("same wall clock, different zone:", dt_eq(datetime(2024,6,20,10,13,42,tzinfo=NYC), datetime(2024,6,20,10,13,42,tzinfo=LAX)))
print("same time, different zone:", dt_eq(datetime(2024,6,20,10,13,42,tzinfo=NYC), datetime(2024,6,20,7,13,42,tzinfo=LAX)))
print("same time, same zone, different fold:", dt_eq(datetime(2024,11,3,1,30, fold = 0, tzinfo = NYC), datetime(2024,11,3,1,30, fold = 1, tzinfo = NYC)))
print("same time, equal zone, different fold:", dt_eq(datetime(2024,11,3,1,30, fold = 0, tzinfo = NYC), datetime(2024,11,3,1,30, fold = 1, tzinfo = NYC1)))

same time, same zone: True
same wall clock, different zone: False
same time, different zone: True
same time, same zone, different fold: False
same time, equal zone, different fold: False


### 5 - Hashing
Since python datetime objects are hashable, they have to guarantee that `dt.__hash__` is constant and (more importantly) the value of `dt1 == dt2` must be constant for any two datetimes. Since time equality is (sometimes) determined according to utc timestamp ([here](https://blog.ganssle.io/articles/2018/02/a-curious-case-datetimes.html#footnote-reference-3)), this also must guarantee that if two timestamps are equal, they must stay equal.

This is an issue when looking at dates in the future because the timezone details might change. For example, if the US removes daylight savings time, we have two options for what a future datetime object that used to be in DST could do:
 * Not change their UTC timestamp and be inaccurate about what UTC time this zoned time refers to
 * Change their UTC timestamp and break equality

Neither of which is good.

As far as I can tell, dateutil.tz chooses the first approach, but I would need to check other libraries to really know if the equality guarantee works with the current datetime packages.